<h1 align = center>Union de data/Merge</h1>

In [1]:
import pandas as pd

In [2]:
games_data = pd.read_parquet('games.parquet')
items_data = pd.read_parquet('items.parquet')
reviews_data= pd.read_parquet('reviews.parquet')

<h2  align = center>Preparacion para la union de los dataframe <sub>/ Data preparation for merging DataFrames<sub></h2>

<h3  align = left> Dataframe item </h3>

In [3]:
# Columnas de importancia de la data de items / Important columns in item data
items_data = items_data[['user_id','item_id','playtime_forever']]

# Creacion de id único para la data de items / Creating unique Id's for item data
items_data['item_id'] = items_data['item_id'].astype(str)
items_data['id'] = items_data['user_id'] + items_data['item_id']

<h3  align = left> Dataframe reviews </h3>

In [4]:
# Cambio de tipo de dato a la columna 'item_id' del DataFrame 'games_data' / "Changing the data type of the 'item_id' column in the 'games_data' DataFrame
games_data['item_id'] = games_data['item_id'].astype(str)

# generar un identificador único en el DataFrame 'user_reviews_data'
reviews_data['item_id'] = reviews_data['item_id'].astype(str)
reviews_data['id'] = reviews_data['user_id'] + reviews_data['item_id']

In [66]:
reviews_data

,user_id,item_id,recommend,year,sentiment_analysis,id
0,76561197970982479,1250,1,2011,2,765611979709824791250
1,76561197970982479,22200,1,2011,2,7656119797098247922200
2,76561197970982479,43110,1,2011,2,7656119797098247943110
3,js41637,251610,1,2014,2,js41637251610
4,js41637,227300,1,2013,2,js41637227300
...,...,...,...,...,...,...
59280,wayfeng,730,1,2015,1,wayfeng730
59283,76561198251004808,253980,1,2015,2,76561198251004808253980
59293,72947282842,730,1,2015,0,72947282842730
59295,ApxLGhost,730,1,2015,2,ApxLGhost730


<h3  align = left> Union <sub>/ Merge </h3>

In [5]:
# Union de dataframes reviews_data y games_data por el 'item_id' y eliminacion de nulos / "Merging dataframes reviews_data and games_data on 'item_id' and removing null values
merged_df = reviews_data.merge(games_data, on='item_id', how='left')
merged_df.dropna(inplace=True)

# Generar dataframe unico uniendo 'items_data' con 'merge_df' por el identificador unico 'id' de nombre 'steam_data' / Generar dataframe unico uniendo 'items_data' con 'merge_df' por el identificador unico 'id' de nombre 'steam_data'
steam_data = items_data.merge(merged_df, on = 'id')

# Renombrando columnas del dataframe 'steam_data' / renaming columns in the 'steam_data' dataframe
steam_data = steam_data.rename(columns={'user_id_x':'user_id'})
steam_data = steam_data.rename(columns={'item_id_x':'item_id'})

# Eliminando columnas redundantes del dataframe 'steam_data' / Removing redundant columns from the 'steam_data' dataframe
steam_data.drop('user_id_y', axis='columns', inplace=True)
steam_data.drop('item_id_y', axis='columns', inplace=True)

# Cambio de tipos de datos de algunas columnas / Changing data types of some columns
steam_data['release_year'] = steam_data['release_year'].astype(int)
steam_data['year'] = steam_data['year'].astype(int)
steam_data['playtime_forever'] = steam_data['playtime_forever'].astype(int)

# Seleccionando columnas definitivas para el analisis / Selecting definitive columns for analysis
steam_data = steam_data[['id','user_id','item_id','title','genre','developer','release_year','price','recommend','year','sentiment_analysis','playtime_forever']]

<h4>Convercion de archivos a parquet<sub>/Converting files to Parquet format<sub></h4>

In [6]:
# Exportando archivo separado por comas de nombre 'data_steam' / exporting a comma-separated file named 'data_steam
steam_data.to_parquet('data_steam.parquet', index=False)